# Solucao Extracao pdf v0

0_solucao_extracao_pdf_v0.ipynb

## 1. Importaçao de modulos e config

<h3> Configuracoes <mark> blocos </mark> </h3>

In [1]:
import os
import csv

import json
import shutil
from io import StringIO
from pathlib import Path
from PIL import Image
from pdf2image import convert_from_path

import matplotlib.pyplot as plt

import cv2
from PIL import Image
import pytesseract

import fitz  # Módulo PyMuPDF
import re
from unidecode import unidecode
from unicodedata import normalize
import PyPDF2

import pandas as pd

from PIL import ImageFont
from PIL import Image, ImageDraw



# path para documentos PDF (omelhor se estiverem dentro de um unico diretorio)
root_pdf_path = "novo_modelo/data_pdf"


# path para documentos PDF que podem estar aguardando para serem processados
root_pdf_aguardando_path = "novo_modelo/pdf_aguardando_processar"

# path para documentos PDF externos para serem processados
root_external_pdf_path = "content_from_pdftool/data/data_pdf/NF_para_processamento/NFRJ_PDF_para _ocr"

# path para documentos PDF PESQUISAVEIS externos para serem processados
root_external_pdf_pesquisavel_path = "content_from_pdftool/data/data_pdf/NF_processadas/NFRJ/fwdnotasfiscaisemitidaslmpadalegal"


# path para modelos
nf_model_path = "novo_modelo/modelos/frames_nf_v5_mage1.xlsx"

# path para imagem padrao
image_resized_path = 'novo_modelo/images/processadas'


# path para NFs processadas
nf_processada_path = "novo_modelo/processados"


model_dict_path = "novo_modelo/modelos/models.csv"


# modelo (fixo neste momento)
model = "mage_1"

poppler_path = "/home/dani-boy/miniconda3/envs/tables-detr/bin"




tgt_imagens = "novo_modelo/images"

# path para datasets CNAE e Itens de Serviço
nf_datasets_path = "novo_modelo/datasets"

# path para config Tesseract
tessdata_dir_config = '--tessdata-dir "/home/dani-boy/miniconda3/envs/tables-detr/share/tessdata/" --user-patterns "novo_modelo/modelos/user-patterns2.txt" --dpi 600 --oem 3 --psm 6'

## 2. Funçoes gerais de imagem e consistencias

In [11]:
# 1. Funcao de conversao e resize do documento
def convertResize(doc2convert, document_path, image_resized_path):
    
    """# 1. remocao do sufixo .pdf
    if doc2convert.split(".")[1].islower():
        nameImage= doc2convert.removesuffix(".pdf")
    else:
        nameImage= doc2convert.removesuffix(".PDF")"""
    
    # 2. construo um novo nome para o documento imagem
    image_resized_name = os.path.join(f'{image_resized_path}/{str(doc2convert)}.jpg')
    
    # 3. Conversao para imagem
    pages = convert_from_path(document_path, 500, poppler_path=poppler_path)
    
    # 4. Verifica se ha mais que uma pagina
    if len(pages) > 1:
        raise ValueError("Erro, documento com mais de uma página")
    else:
        # 5. Iterar pelas páginas e redimensionar
        resized_pages = []
        for page in pages:
            resized_page = page.resize((2067, 2923))
            resized_pages.append(resized_page)
            
        resized_pages[0].save(image_resized_name, 'JPEG')
        
    return resized_pages[0], image_resized_name

# 2. Leitura do arquivo CSV e criação do dicionário modelos
def create_model_dictionary(model_dict_path):
    model_dictionary = {}
    with open(model_dict_path, 'r') as csvfile:
        csvreader = csv.DictReader(csvfile)
        for row in csvreader:
            prefeitura_name = row['prefeitura']
            model_name = row['model']

            if prefeitura_name not in model_dictionary:
                model_dictionary[prefeitura_name] = model_name
            
            #model_dictionary[prefeitura_name].append(model_name)
    
    return model_dictionary

# 3. Cria o dict de modelos
model_dict = create_model_dictionary(model_dict_path) 

# 3. Pesquisa prefeitura no documento
def pequisaModel(image_name):

    # 1. Definindo as coordenadas do frame
    x0 = 406
    y0 = 0
    x1= 1540
    y1 = 380

    # 2. Definir frame_image
    frame_image = image_name.crop((x0, y0, x1, y1))

    # 3. Extraia texto usando OCR com configuração de idioma padrão para este frame
    extracted_text_frame = pytesseract.image_to_string(frame_image, lang='por', config=tessdata_dir_config).strip()

    # 4. Divida o texto por nova linha e mantenha apenas a última parte (assume que o valor está sempre no final)
    values = extracted_text_frame.split('\n')

    # 5. Interacao para pesquisar prefeitura
    for value in values:
        nome_prefeitura_match = re.search(r'PREFEITURA (.+)', value)
        if nome_prefeitura_match:
            nome_prefeitura = "PREFEITURA " + nome_prefeitura_match.group(1) 
            return  nome_prefeitura
   
   
# 5. Ajusta o filename tirando caracteres especiais 
def conv_filename(title):
    
    # Divida o título em nome e extensão
    name, extension = title.rsplit('.', 1) if '.' in title else (title, "")

    # Remova acentos e caracteres especiais do nome
    name = normalize('NFKD', name).encode('ASCII', 'ignore').decode('ASCII')
    
    # Substitua espaços e hífens por sublinhados
    filename = name.replace(' ', '_').replace('-', '_')

    # Remova quaisquer outros caracteres não alfanuméricos, exceto sublinhados
    filename = re.sub(r'[^\w_]', '', filename)

    # Converter para minúsculas
    filename = filename.lower()

    # Adicione a extensão de volta, se houver
    if extension:
        filename += '.' + extension.lower()

    return filename

# 6. Ajusta o filename tirando caracteres especiais e a
def conv_filename_no_ext(title):
    # Divida o título em nome e extensão (mas ignore a extensão)
    name = title.rsplit('.', 1)[0] if '.' in title else title

    # Remova acentos e caracteres especiais do nome
    name = normalize('NFKD', name).encode('ASCII', 'ignore').decode('ASCII')
    
    # Substitua espaços e hífens por sublinhados
    filename = name.replace(' ', '_').replace('-', '_')

    # Remova quaisquer outros caracteres não alfanuméricos, exceto sublinhados
    filename = re.sub(r'[^\w_]', '', filename)

    # Converter para minúsculas
    filename = filename.lower()

    return filename     

# 7. Verifica se PDF e pesquisavel ou nao e grava metadados dele
def is_pdf_searchable(pdf_path):
    try:
        pdf_document = fitz.open(pdf_path)
        is_searchable = all(page.get_text("text") != "" for page in pdf_document)
        dados_pdf = pdf_document.metadata
        pdf_document.close()
        return is_searchable, dados_pdf
    except Exception as e:
        print(f"Erro ao verificar o PDF: {e}")
        return False

#Le a planilha e cria do DF
frames_nf_v3_df = pd.read_excel(nf_model_path)

# Cria dicionários para armazenar diferentes tipos de elementos do modelo
document_info = frames_nf_v3_df[frames_nf_v3_df['type'] == 'document'].iloc[0]
boundaries_info = frames_nf_v3_df[frames_nf_v3_df['type'] == 'boundaries']
sections_info = frames_nf_v3_df[frames_nf_v3_df['type'] == 'section']
frames_info = frames_nf_v3_df[frames_nf_v3_df['type'] == 'frame']
sframe_fields_info = frames_nf_v3_df[frames_nf_v3_df['type'] == 'sframe_field']
field_boxes_info = frames_nf_v3_df[frames_nf_v3_df['type'] == 'field_box']


# Define a mapping for color names to RGB values
color_mapping = {
    "red": (255, 0, 0),
    "purple": (128, 0, 128),
    "orange": (255, 165, 0),
    "green": (0, 128, 50),
    "blue": (0, 0, 255),
    "yellow": (255, 255, 0)
}

# Reload the image to start fresh
image = Image.open(name_image_2work)
draw = ImageDraw.Draw(image)

# Define a font size for the labels using the default PIL font
font_size = 100
#font = ImageFont.load_default()

font = ImageFont.truetype("/usr/share/fonts/truetype/ubuntu/Ubuntu-M.ttf", 30, encoding="unic")

# Update the draw_box function to use the larger font size with the default font
def draw_box(row):
    x0, y0, x1, y1 = row['x0'], row['y0'], row['x1'], row['y1']
    color = color_mapping.get(row['color'], (0, 0, 0)) # Default to black if color not found
    draw.rectangle([x0, y0, x1, y1], outline=color, width=3)
    label = str(row['label']) if pd.notnull(row['label']) else None # Check for missing label
    if label:
        draw.text((x0 + 5, y0 + 5), label, fill=color, font=font)

# Draw the boundaries
#draw_box(boundaries_info)


def draw_box_model(modelo,
                   boundaries_info=None,
                   sections_info=None,
                   frames_info=None,
                   field_boxes_info=None,
                   draw_boundaries=True,
                   draw_sections=True,
                   draw_frames=True,
                   draw_field_boxes=True):
    
    # Draw boundaries if requested
    if draw_boundaries and boundaries_info is not None:
        filtered_boundaries_info = boundaries_info[boundaries_info['model'] == modelo]
        for index, row in filtered_boundaries_info.iterrows():
            draw_box(row)

    # Draw sections if requested
    if draw_sections and sections_info is not None:
        filtered_sections_info = sections_info[sections_info['model'] == modelo]
        for index, row in filtered_sections_info.iterrows():
            draw_box(row)
            
    # Draw frames if requested
    if draw_frames and frames_info is not None:
        filtered_frames_info = frames_info[frames_info['model'] == modelo]
        for index, row in filtered_frames_info.iterrows():
            draw_box(row)
            
    # Draw field boxes if requested
    if draw_field_boxes and field_boxes_info is not None:
        filtered_field_boxes_info = field_boxes_info[field_boxes_info['model'] == modelo]
        for index, row in filtered_field_boxes_info.iterrows():
            draw_box(row)
    
    # Show the image with selected drawings
    image.show()

## 3. Pipeline

<h3>1. Tratando o PDF e seus dados </h3>

In [3]:
# 1. Leitura recursiva de diretorios e arquivos a partir de root

for roots, directories, documents in os.walk(root_pdf_path):
                for document in documents:
                    
                    
                    doc2convert = document
                    document_path_1 = os.path.join(roots, document)
                    
                    print(f'{roots}  | {directories} | {document} | doc2convert: {doc2convert} | document_path_1: {document_path_1}\n')
                    
                    #logging.info("Processando arquivo: %s", document_path_1)
                    
                    
                    print(is_pdf_searchable(document_path_1))
                    #print(document_path, doc2convert)
                    


novo_modelo/data_pdf  | [] | NFS_1156_2023_Ikinha.pdf | doc2convert: NFS_1156_2023_Ikinha.pdf | document_path_1: novo_modelo/data_pdf/NFS_1156_2023_Ikinha.pdf

(True, {'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36', 'producer': 'Skia/PDF m115', 'creationDate': "D:20230801175411+00'00'", 'modDate': "D:20230801175411+00'00'", 'trapped': '', 'encryption': None})


In [4]:
# 2. Ajusta o nome do arquivo tirando caracteres especiais e a extensao
doc2convert_named = conv_filename_no_ext(doc2convert)


# 3. Converte e resize da imagem "on the fly"
image_2work, name_image_2work = convertResize(doc2convert_named, document_path_1, image_resized_path) #Utilizado como origem: document_path_1

#print(name_image_2work) 


# 4. Busco nome da prefeitura e modelo
name_prefeitura = pequisaModel(image_2work)

modelo = model_dict[name_prefeitura]

if model_dict[name_prefeitura]:
    modelo = model_dict[name_prefeitura]
    print(f'\nNome Prefeitura: {name_prefeitura},  modelo: {modelo}') 
else:
    modelo = model_padrao
    message = f'\nmodelo nao encontrado para prefeitura {name_prefeitura}, sera utilizado o modelo {modelo}'   
    print(message)     


Nome Prefeitura: PREFEITURA MUNICIPAL DE MAGE,  modelo: mage_1


<h3>2. Buscando o proximo PDF e efetuando as mudancas de diretorios </h3>

In [ ]:
tipo_pdf = "pequisavel"

In [ ]:
tipo_pdf = "nao pequisavel"

In [ ]:
# 4.5 Definir o tipo de PDF a buscar  (TESTE APENAS)
#source_directory = root_external_pdf_path

#print(source_directory )
destination_directory = root_pdf_path
if tipo_pdf == "pequisavel":
    source_directory = root_external_pdf_pesquisavel_path
    print(f'\nTipo PDF a utilizar: {tipo_pdf} | source_directory: {source_directory} \n\nDocumento que sera movido: {document_path_1}\n')
    
else:
    source_directory = root_external_pdf_path

#print(f'\nTipo PDF a utilizar: {tipo_pdf} | source_directory: {source_directory} \n\nDocumento que sera movido: {document_path_1}\n')   


# 5. MOVER PDF para diretorio de processados
source_path = document_path_1
destination_path = os.path.join(f'{nf_processada_path}/{str(doc2convert)}')

print(f'\nsource_path: {source_path} | destination_path: {destination_path} \n') 


shutil.move(source_path, destination_path)

# 6. ***DELETAR**** o arquivo de imagem gerado
image_path_to_delete = name_image_2work
os.remove(image_path_to_delete)


# Verifique se o diretório de destino está vazio
if os.listdir(destination_directory):
    raise Exception("O diretório de destino não está vazio!")

for roots, directories, documents in os.walk(source_directory):
    # Filtre os documentos para incluir apenas aqueles com extensão .pdf
    pdf_files = [doc for doc in documents if doc.lower().endswith('.pdf')]
    
    if pdf_files: # Verifique se há algum arquivo PDF no diretório
        first_pdf_file = pdf_files[0] # Obtenha o primeiro arquivo PDF
        source_path = os.path.join(roots, first_pdf_file) # Construa o caminho completo para o primeiro arquivo PDF
        print(f"Found the first PDF file: {source_path}")
        
        # Mova o arquivo para o diretório de destino
        shutil.move(source_path, destination_directory)
        
        break # Saia do loop após encontrar o primeiro arquivo PDF

## 4. Implementaçao do Pipeline de Extracao & Validaçoes

### 4.1. Leitura recursiva do diretorio: root_pdf_path

In [ ]:
# 1. Leitura recursiva de diretorios e arquivos a partir de root

for roots, directories, documents in os.walk(root_pdf_path):
                for document in documents:
                    
                    
                    doc2convert = document
                    document_path_1 = os.path.join(roots, document)
                    
                    print(f'{roots}  | {directories} | {document} | doc2convert: {doc2convert} | document_path_1: {document_path_1}\n')
                    
                    #logging.info("Processando arquivo: %s", document_path_1)
                    
                    
                    print(is_pdf_searchable(document_path_1))
                    #print(document_path, doc2convert)

### 4.2. Ajustes, criaçao imagem e busca do modelo

In [ ]:
# 2. Ajusta o nome do arquivo tirando caracteres especiais e a extensao
doc2convert_named = conv_filename_no_ext(doc2convert)


# 3. Converte e resize da imagem "on the fly"
image_2work, name_image_2work = convertResize(doc2convert_named, document_path_1, image_resized_path) #Utilizado como origem: document_path_1

#print(name_image_2work) 


# 4. Busco nome da prefeitura e modelo
name_prefeitura = pequisaModel(image_2work)

modelo = model_dict[name_prefeitura]

if model_dict[name_prefeitura]:
    modelo = model_dict[name_prefeitura]
    print(f'\nNome Prefeitura: {name_prefeitura},  modelo: {modelo}') 
else:
    modelo = model_padrao
    message = f'\nmodelo nao encontrado para prefeitura {name_prefeitura}, sera utilizado o modelo {modelo}'   
    print(message)     

### 4.3. Draw Boxes

In [ ]:
# To draw only boundaries and sections:
draw_box_model(modelo, boundaries_info, sections_info, draw_frames=False, draw_field_boxes=False)


In [ ]:
# To draw everything
draw_box_model(modelo, boundaries_info, sections_info, frames_info, field_boxes_info)


In [ ]:
# To draw only field boxes:
draw_box_model(modelo, field_boxes_info=field_boxes_info, draw_boundaries=False, draw_sections=False, draw_frames=False)


## 4.5. Modelo de Extracao

### 4.5.1. Funcoes de Extracao

In [33]:
def extract_text_from_frame(image, coordinates, config):
    x0, y0, x1, y1 = coordinates
    frame_image = image.crop((x0, y0, x1, y1))
    extracted_text = pytesseract.image_to_string(frame_image, lang='por', config=config).strip()
    return extracted_text

def process_line(value, reference, label):
    name_match = re.search(fr'{reference} (.+)', value)
    if name_match:
        extracted_value = reference + " " + name_match.group(1)
        return {label: extracted_value}
    return None


def extract_fields_box(modelo, father_value, section):

    data_box_valores = {}
    data_box_valores['secao'] = section
    filtered_boxes_info = field_boxes_info[(field_boxes_info['father'] == father_value) & (field_boxes_info['model'] == modelo)]
    # Iterate nas informações dos boxes de fields e extraia o texto de cada field
    for index_field, row_field in filtered_boxes_info.iterrows():
        x0, y0, x1, y1 = row_field['x0'], row_field['y0'], row_field['x1'], row_field['y1']
        extracted_text_box = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
        #print("{:<5} {:<10} {:<30} {:<20} {:<20} {:<7} {:<7} {:<7} {:<7}".format(row_field['seq'], row_field['model'], row_field['father'], row_field['label'], row_field['reference'], row_field['x0'], row_field['y0'], row_field['x1'], row_field['y1'] ))
        # Divida o texto por nova linha e mantenha apenas a última parte (assume que o valor está sempre no final)
        value = extracted_text_box.split('\n')[-1]
        # Remova qualquer espaço em branco à esquerda ou à direita
        value = value.strip()
        if row_field['t_value'] == 'number':
            # Formate o valor usando a função format_number
            #print("vou verificar valor")
            value = format_number(value)
            #print(value)
        # Armazene o texto extraído com o rótulo correspondente
        label = row_field['label']
        data_box_valores[label] = value
        
    return data_box_valores

# Funçao de formatacao de numeros
def format_number(number_str):
    number_str = number_str.replace('R$', '').replace('.', '').replace(',', '.')
    if '%' in number_str:
        number_str = number_str.replace('%', '')
        return float(number_str)  # multiplica por 100 para fields %
    return float(number_str)

    # Dicionário para armazenar o texto final processado para cada field
    final_extracted_data = {}

    # Itere pelos dados de texto extraídos e processe o texto para manter apenas os valores
    for reference, extracted_text in extracted_text_data_default_lang.items():
        # Divida o texto por nova linha e mantenha apenas a última parte (assume que o valor está sempre no final)
        value = extracted_text.split('\n')[-1]
        # Remova qualquer espaço em branco à esquerda ou à direita
        value = value.strip()
        # Formate o valor usando a função format_number
        value = format_number(value)
        # Armazene o valor processado no dicionário final
        final_extracted_data[reference] = value

    # Display the final extracted data for the first few fields
    list(final_extracted_data.items())[:15]


def extract_fields_prestador_cnpj(text): # Função para extrair campos e valores dentro de um retângulo
    nf_data_prestador_cnpj = {}
    
    nf_data_prestador_cnpj['secao'] = "2. PRESTADOR DE SERVIÇO"
    
    
    # Extrair CPF/CNPJ com máscara 1
    if "CPF/CNPJ:" in text:
        cpf_cnpj_formatado_match = re.search(r'(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})', text)
        if cpf_cnpj_formatado_match:
                        nf_data_prestador_cnpj['cpf_cnpj_com_mascara'] = cpf_cnpj_formatado_match.group(1)
                        nf_data_prestador_cnpj['cpf_cnpj_sem_mascara'] = re.sub(r'\D', '', cpf_cnpj_formatado_match.group(1))


    # Extrair Telefone
    telefone_str = None
    
    #telefone_match = re.search(r'Telefone:\s+([0-9.\s-])', text)
    telefone_match = re.search(r'Telefone:\s+([0-9.\s-]+)', text)
    if telefone_match: 
        telefone_str = telefone_match.group(1)
        # Remover quebras de linha
        telefone_str = telefone_str.replace('.', '')
        telefone_str = telefone_str.replace('\n', '')
                
        nf_data_prestador_cnpj['telefone'] = telefone_str
    else:
        nf_data_prestador_cnpj['telefone'] = None   
    
    
    return nf_data_prestador_cnpj 


def extract_fields_tomador_cnpj(text):
    nf_data_tomador_cnpj = {}
    
    
    nf_data_tomador_cnpj['secao'] = "3.TOMADOR DE SERVIÇO"
    
    
    # Extrair CPF/CNPJ com máscara 1
    if "CPF/CNPJ:" in text:
        cpf_cnpj_formatado_match = re.search(r'(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})', text)
        if cpf_cnpj_formatado_match:
                        nf_data_tomador_cnpj['cpf_cnpj_com_mascara'] = cpf_cnpj_formatado_match.group(1)
                        nf_data_tomador_cnpj['cpf_cnpj_sem_mascara'] = re.sub(r'\D', '', cpf_cnpj_formatado_match.group(1))

    
    # Extrair Telefone
    telefone_match = re.search(r'Telefone:\s+(.+)', text)
    if telefone_match:
        telefone_str = telefone_match.group(1)
        if telefone_str == 'Inscrição Estadual:':
            nf_data_tomador_cnpj['telefone'] = None  # Valor padrão quando não há correspondência
        elif telefone_str == '':
            nf_data_tomador_cnpj['telefone'] = None  # Valor padrão quando não há correspondência
                    
        else:    
            nf_data_tomador_cnpj['telefone'] = telefone_match.group(1)
            
    
    
    # Extrair Inscrição Municipal
    inscricao_municipal_match = re.search(r'Inscrição Municipal:\s+(.+)', text)
    if inscricao_municipal_match:
        inscricao_municipal_str = inscricao_municipal_match.group(1)
        if inscricao_municipal_str == "Telefone:": 
            nf_data_tomador_cnpj['inscricao_municipal'] = None
        else:    
            nf_data_tomador_cnpj['inscricao_municipal'] = inscricao_municipal_str
    
    insc_municipal_match = re.search(r'INSC:MUNICIPAL:\s+(.+)', text)
    if insc_municipal_match:
        insc_municipal_str = insc_municipal_match.group(1)
        if insc_municipal_str == "Telefone:":
            nf_data_tomador_cnpj['inscricao_municipal'] = None
        else:    
            nf_data_tomador_cnpj['inscricao_municipal'] = insc_municipal_str
    else:
        nf_data_tomador_cnpj['inscricao_municipal'] = None
            
    
    return nf_data_tomador_cnpj



def extract_fields_tomador_inscricao(text):
    nf_data_tomador_inscricao = {}
    
    # Extrair RG    
    rg_match = re.search(r'RG:\s+(.+)', text)   
    if rg_match:
        rg_str = rg_match.group(1)
        if rg_str == 'Inscrição Estadual:':
            nf_data_tomador_inscricao['rg'] = None  # Valor padrão quando não há correspondência
        else:    
            nf_data_tomador_inscricao['rg'] = rg_match.group(1)  
 
        
                
    # Extrair Inscrição Estadual
    inscricao_estadual_match = re.search(r'Inscrição Estadual:\s+(.+)', text)
    if inscricao_estadual_match:
        inscricao_estadual_str = inscricao_estadual_match.group(1)
        if inscricao_estadual_str == '':
            nf_data_tomador_inscricao['inscricao_estadual'] = None
        else:    
            nf_data_tomador_inscricao['inscricao_estadual'] = inscricao_estadual_match.group(1)   
    else: 
        nf_data_tomador_inscricao['inscricao_estadual'] = None           

    return nf_data_tomador_inscricao



def extract_fields_tomador_dados(text):
    
    nf_data_tomador = {}

    # Dividir o texto em linhas
    linhas = text.split('\n')

    # Inicializar variáveis para armazenar os valores
    nome_razao_social = None
    endereco = None
    email = None

    # Iterar pelas linhas para identificar os campos e valores
    i = 0
    while i < len(linhas):
        linha = linhas[i]
        if "Nome/Razão Social:" in linha:
            nome_razao_social = linhas[i + 1].strip()
            i += 2
        elif "Endereço:" in linha:
            endereco = linhas[i + 1].strip()
            i += 2
        elif "E-mail:" in linha:
            email = linhas[i + 1].strip()
            i += 2
        else:
            i += 1


    nf_data_tomador['razao_social'] = nome_razao_social
    nf_data_tomador['endereco'] = endereco
    nf_data_tomador['email'] = email
    
    return nf_data_tomador


### secao: 1 - CABECALHO

In [ ]:
data_cabecalho = {}
# Filtrar o DataFrame para incluir apenas linhas onde a coluna "model" oriundo de: modelo
filtered_frames_info = frames_info[frames_info['model'] == modelo]

for index_frame, row_frame in filtered_frames_info.iterrows():
    frame_model = row_frame['model']
    if frame_model == model:
        frame_father = row_frame['label']
        # ... Select specific frames ...
        if frame_father == "1_frame_prefeitura_nf" or frame_father == "1_frame_dados_nf":
            
            # Extrai coordenadas para recorte
            x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
            extracted_text_frame = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
            
            seq = row_frame['seq']
            #print(f'\nLoop 1: {frame_model}, seq.: {seq}, row_frame[label]: {frame_father}\n')
            data_cabecalho['secao'] = "1 - CABECALHO"
            
            for index_field, row_field in sframe_fields_info.iterrows():
                item_son = row_field['father']
                frame_father = row_frame['label']
                if item_son == frame_father:

                    model_value = row_field['model']
                    type_value = row_field['type']
                    label_value = row_field['label']
                    reference_value = row_field['reference']
                    seq_value = row_field['seq']
                    father_value = row_field['father']
                    # ... Select specific fields ...
                    if label_value == "nome_prefeitura" and model_value == frame_model: 
                        #print(f'  Loop 2: {model_value}, seq.: {seq_value}, type_value: {type_value}, label_value: {label_value}')
                        values = extracted_text_frame.split('\n')
                        for value in values:
                            result = process_line(value, reference_value, label_value)
                            if result:
                                data_cabecalho.update(result)
                                
                    elif label_value == "secretaria" and model_value == frame_model: 
                        values = extracted_text_frame.split('\n')
                        for value in values:
                            result = process_line(value, reference_value, label_value)
                            if result:
                                data_cabecalho.update(result)
                                
                    elif label_value == "tipo_nota_fiscal" and model_value == frame_model:
                        values = extracted_text_frame.split('\n')
                        for value in values:
                            result = process_line(value, reference_value, label_value)
                            if result:
                                data_cabecalho.update(result)
                    
                    #Extraçao de Dados da NF            
                    elif father_value == "1_frame_dados_nf" and model_value == frame_model:
                        values = extracted_text_frame.split('\n')
                        data_list = [item for item in values if item != '']
                        
                        data_dict = {}
                        for i in range(0, len(data_list), 2):
                            key = data_list[i]
                            value = data_list[i+1]
                            data_dict[key] = value

                        # You can now access the values using the corresponding labels
                        data_cabecalho['numero_nota_fiscal'] = data_dict['Número da Nota:']
                        data_cabecalho['competencia'] = data_dict['Competência:']
                        data_cabecalho['dt_hr_emissao'] = data_dict['ata e Hora da Emissão:']
                        data_cabecalho['codigo_verificacao'] = data_dict['Código Verificação:']

### secao: 2 - PRESTADOR DE SERVIÇO

In [ ]:
data_prestador = {}

# Filtrar o DataFrame para incluir apenas linhas onde a coluna "model" oriundo de: modelo
filtered_frames_info = frames_info[frames_info['model'] == modelo]

for index_frame, row_frame in filtered_frames_info.iterrows():
    frame_model = row_frame['model']
    if frame_model == model:
        frame_father = row_frame['label']
        # ... Select specific frames ...
        if frame_father == "2_frame_cnpj_prestador":
            
            # Extrai coordenadas para recorte
            x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
            extracted_text_frame = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
            data_prestador = extract_fields_prestador_cnpj(extracted_text_frame)
            
        elif frame_father == "2_frame_inscricao_prestador":
            # Extrai coordenadas para recorte
            x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
            extracted_text_frame = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
            # 1. Prrestador de Servico - INSCRICAO ESTADUAL/MUNICIPAL
            nf_data_prestador_incricao = {}

            # Dividir o texto em linhas
            linhas = extracted_text_frame.split('\n')

            # Inicializar variáveis para armazenar os valores
            inscricao_municipal = None
            inscricao_estadual = None

            # Iterar pelas linhas para identificar os campos e valores
            i = 0
            while i < len(linhas):
                linha = linhas[i]
                if "Inscrição Municipal:" in linha:
                    inscricao_municipal = linhas[i + 1].strip()
                    i += 2
                elif "Inscrição Estadual:" in linha: 
                    inscricao_estadual = linhas[i + 1].strip()   
                    i += 2
                else:
                    i += 1
                    
            if inscricao_municipal == "":
                inscricao_municipal = "None"        
            if inscricao_estadual == "":
                inscricao_estadual = "None"

            data_prestador['inscricao_municipal'] = inscricao_municipal
            data_prestador['inscricao_estadual'] = inscricao_estadual
            
        elif frame_father == "2_frame_dados_prestador":
            # Extrai coordenadas para recorte
            x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
            extracted_text_frame = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
            tipo = None

            if "\n\n" in extracted_text_frame:
                linhas = extracted_text_frame.split('\n\n')
                tipo = 2
                
            else:
                linhas = extracted_text_frame.split('\n')    
                tipo = 1
            
            # Inicializar variáveis para armazenar os valores
            nome_razao_social = None
            nome_fantasia = None
            endereco = None
            email = None

            # Iterar pelas linhas para identificar os campos e valores
            i = 0
            while i < len(linhas):
                linha = linhas[i]
                if "Nome/Razão Social:" in linha:
                    if tipo == 2:
                        texto1 = linhas[i + 1].strip()
                        texto2 = texto1.split('\n')
                        nome_razao_social = texto2[i].strip()
                    else:
                        nome_razao_social = linhas[i + 1].strip()   

                    i += 2
                elif "Nome de Fantasia:" in linha:
                    nome_fantasia = linhas[i + 1].strip()
                    i += 2
                elif "Endereço:" in linha:
  
                    if tipo == 2:
                        texto3 = linhas[i + 1].strip()
                        texto4 = texto3.split('\n')
                        
                        endereco = texto4[0].strip()
                    else:
                        endereco = linhas[i + 1].strip()      
                    i += 2
                elif "E-mail:" in linha:
                    email = linhas[i + 1].strip()
                    i += 2
                else:
                    i += 1

            data_prestador['razao_social'] = nome_razao_social
            data_prestador['nome_fantasia'] = nome_fantasia
            data_prestador['endereco'] = endereco
            data_prestador['email'] = email
                    
                        

### secao: 3 - TOMADOR DE SERVIÇO, 
            4 - DESCRIMINACAO DOS SERVIÇOS,  
            5 - VALOR TOTAL, data_tomador, 
            6 - CNAE e Item da Lista de Serviços

In [ ]:
data_tomador = {}
data_servico = {}
data_valor_total = {}
data_CNAE = {}


# Filtrar o DataFrame para incluir apenas linhas onde a coluna "model" oriundo de: modelo
filtered_frames_info = frames_info[frames_info['model'] == modelo]

for index_frame, row_frame in filtered_frames_info.iterrows():
    frame_model = row_frame['model']
    if frame_model == model:
        frame_father = row_frame['label']
        # ... Select specific frames ...
        if frame_father == "3_frame_cnpj_tomador":
            # Extrai coordenadas para recorte
            x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
            extracted_text_frame = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
            nf_data_tomador_cnpj = extract_fields_tomador_cnpj(extracted_text_frame)
            
        elif frame_father == "3_frame_inscricao_tomador":
            # Extrai coordenadas para recorte
            x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
            extracted_text_frame = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
            nf_data_tomador_inscricao = extract_fields_tomador_inscricao(extracted_text_frame)
        
        elif frame_father == "3_frame_dados_tomador":
            # Extrai coordenadas para recorte
            x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
            extracted_text_frame = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
            nf_data_tomador = extract_fields_tomador_dados(extracted_text_frame)
        
        elif frame_father == "4_frame_descricao_totais":
            data_servico['secao'] = "4. DESCRIMINACAO DOS SERVIÇOS"
            # Extrai coordenadas para recorte
            x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
            extracted_text_frame = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
            text = extracted_text_frame.replace('\n', ' ')
            label = "DISCRIMINAÇÃO DOS SERVIÇOS"
            if text.startswith(label):
                text = text[len(label):].strip()
            data_servico['discriminacao_servicos'] = text 
        
        elif frame_father == "4_frame_valor_total":
            data_valor_total['secao'] = "5. VALOR TOTAL" 
            # Extrai coordenadas para recorte
            x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
            extracted_text_frame = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
            
            valor_total_match = re.search(r'R\$ ([\d,.]+)', extracted_text_frame)
            if valor_total_match:
                valor_total_sem_formatacao = valor_total_match.group(1).replace('.', '').replace(',', '.')
                data_valor_total['valor_total_nota'] = float(valor_total_sem_formatacao)
                
        elif frame_father == "4_frame_cnae_itens_servico":
            data_CNAE['secao'] = "6. CNAE e Item da Lista de Serviços"
            x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
            extracted_text_frame = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
            
            
            linhas = extracted_text_frame.split('\n')
            i = len(linhas)
            texto_cnae = linhas[0].replace(' . .', '')
            cnae = re.sub(r'^CNAE - ', '', texto_cnae, count=1)


            # Item da Lista de Serviços 
            texto_novo = linhas[1]
            texto_novo = texto_novo.replace('\n', '')

            item_lista_servico = re.sub(r'^Item da Lista de Serviços - ', '', texto_novo, count=1)

            data_CNAE['cnae'] = cnae
            data_CNAE['item_lista_servicos'] = item_lista_servico
                 
            

data_tomador.update(nf_data_tomador_cnpj) 
data_tomador.update(nf_data_tomador_inscricao)
data_tomador.update(nf_data_tomador)         

In [ ]:
# Assume extracted_cnae contains the extracted CNAE code or description from the document
extracted_cnae = 'some_value'

# Search the DataFrame for the extracted CNAE information (adjust the column name as needed)
matching_row = cnae_df[cnae_df['cnae_column'] == extracted_cnae]

# Use the values from the matching row
if not matching_row.empty:
    whole_value = matching_row.iloc[0] # Adjust to get the specific value you need
    # Process the whole_value as needed
else:
    print('CNAE not found in repository')

### secao: 7 - VALORES E IMPOSTOS, 9 - OUTRAS INFORMAÇOES / CRITICAS

In [34]:
# data_valores
data_valores = {}
father_value = "5_frame_valores_impostos"
section = "7. VALORES E IMPOSTOS"

result = extract_fields_box(modelo, father_value, section)
if result:
    data_valores.update(result)
    
data_valores    

{'secao': '7. VALORES E IMPOSTOS',
 'valor_servicos': 1843.75,
 'valor_deducao': 0.0,
 'desc_incond': 0.0,
 'base_calculo': 1843.75,
 'aliquota': 5.0,
 'valor_iss': 92.19,
 'valor_iss_retido': 0.0,
 'desc_cond': 0.0,
 'valor_pis': 0.0,
 'valor_cofins': 0.0,
 'valor_ir': 0.0,
 'valor_inss': 0.0,
 'valor_csll': 0.0,
 'outras_retencoes': 0.0,
 'valor_liquido': 1843.75}

In [35]:
# data_outras_informacoes
data_outras_informacoes = {}
father_value = "5_frame_inf_criticas"
section = "9. OUTRAS INFORMAÇOES / CRITICAS"

result = extract_fields_box(modelo, father_value, section)
if result:
    data_outras_informacoes.update(result)
    
data_outras_informacoes    

{'secao': '9. OUTRAS INFORMAÇOES / CRITICAS',
 'exigibilidade_iss': 'Exigivel',
 'regime_tributacao': 'Sociedade Limitada',
 'simples_nacional': 'Sim (5%)',
 'issqn_retido': 'Não',
 'local_pretacao_servico': 'Magé - RJ',
 'local_incidencia': 'Magé - RJ'}

### Informacoes dos DFSs

In [ ]:
frames_info.head()

sframe_fields_info.head()

field_boxes_info.head()

In [15]:
frames_info.head()

,model,seq,type,color,box,t_value,father,label,reference,x0,y0,x1,y1,Largura,Altura,%
3,mage_1,4,frame,purple,yes,NaN,1_section_cabecalho_nf,1_frame_prefeitura_nf,NaN,406.0,0.0,1540.0,380.0,1030.0,380.0,NaN
7,mage_1,8,frame,purple,yes,NaN,1_section_cabecalho_nf,1_frame_dados_nf,NaN,1633.0,0.0,2067.0,380.0,434.0,380.0,NaN
13,mage_1,14,frame,purple,yes,NaN,2_section_prestador_servico,2_frame_cnpj_prestador,NaN,0.0,380.0,550.0,555.0,440.0,170.0,NaN
17,mage_1,18,frame,purple,yes,NaN,2_section_prestador_servico,2_frame_inscricao_prestador,NaN,550.0,380.0,1357.0,555.0,917.0,170.0,NaN
20,mage_1,21,frame,purple,yes,NaN,2_section_prestador_servico,2_frame_dados_prestador,NaN,0.0,552.0,2067.0,785.0,2067.0,235.0,NaN


In [13]:
sframe_fields_info.head()

,model,seq,type,color,box,t_value,father,label,reference,x0,y0,x1,y1,Largura,Altura,%
4,mage_1,5,sframe_field,NaN,no,string,1_frame_prefeitura_nf,nome_prefeitura,PREFEITURA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,mage_1,6,sframe_field,NaN,no,string,1_frame_prefeitura_nf,secretaria,SECRETARIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,mage_1,7,sframe_field,NaN,no,string,1_frame_prefeitura_nf,tipo_nota_fiscal,NOTA FISCAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,mage_1,9,sframe_field,NaN,no,string,1_frame_dados_nf,numero_nota_fiscal,Número da Nota:,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,mage_1,10,sframe_field,NaN,no,string,1_frame_dados_nf,competencia,Competência:,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
field_boxes_info.head()

,model,seq,type,color,box,t_value,father,label,reference,x0,y0,x1,y1,Largura,Altura,%
48,mage_1,49,field_box,orange,yes,number,5_frame_valores_impostos,valor_servicos,VALOR SERVIÇOS:,344.0,1950.0,575.0,2116.0,231.0,166.0,NaN
49,mage_1,50,field_box,orange,yes,number,5_frame_valores_impostos,valor_deducao,VALOR DEDUÇÃO:,583.0,1950.0,793.0,2116.0,210.0,166.0,NaN
50,mage_1,51,field_box,orange,yes,number,5_frame_valores_impostos,desc_incond,DESC. INCOND:,794.0,1950.0,977.0,2116.0,183.0,166.0,NaN
51,mage_1,52,field_box,orange,yes,number,5_frame_valores_impostos,base_calculo,BASE DE CÁLCULO:,979.0,1950.0,1163.0,2116.0,184.0,166.0,NaN
52,mage_1,53,field_box,orange,yes,number,5_frame_valores_impostos,aliquota,ALÍQUOTA:,1170.0,1950.0,1340.0,2116.0,170.0,166.0,NaN


In [ ]:
# data_dados_complementares
data_dados_complementares = {}
father_value = "5_frame_dados_complementares"
section = "8. DADOS COMPLEMENTARES"

#result = extract_fields_box(modelo, father_value, section)
#if result:
    #data_valores.update(result)
    
#data_valores    

In [ ]:
# data_observacao
data_observacao = {}
father_value = "5_frame_observacao"
section = "10. OBSERVACOES"

#result = extract_fields_box(modelo, father_value, section)
#if result:
    #data_valores.update(result)
    
#data_valores    

### Filtros

In [ ]:
# 1. Filtrando o frames_info para buscar os dados de corte
filtered_frames_info = frames_info[(frames_info['label'] == father_value) & (frames_info['model'] == modelo)]

In [ ]:
# 2. Filtrando o sframe_fields_info para buscar os dados dos campos que estao nos frames
filtered_sframe_fields_info = sframe_fields_info[(sframe_fields_info['father'] == father_value) & (sframe_fields_info['model'] == modelo)]

In [ ]:
# 3. Filtrando o field_boxes_info para buscar os dados de corte dos campos com BOX
filtered_boxes_info = field_boxes_info[(field_boxes_info['father'] == father_value) & (field_boxes_info['model'] == modelo)]

### Funcao: frames_info -> sframe_fields_info

In [25]:
def extract_fields_frame(modelo, father_value, section):
    

    frame_label = father_value
    print(section)
    
    # 1. Filtrando o frames_info para buscar os dados de corte
    filtered_frames_info = frames_info[(frames_info['label'] == frame_label) & (frames_info['model'] == modelo)]
    
    # 2. Filtrando o sframe_fields_info para buscar os dados dos campos que estao nos frames
    filtered_sframe_fields_info = sframe_fields_info[(sframe_fields_info['father'] == father_value) & (sframe_fields_info['model'] == modelo)]
    
    for index_frame, row_frame in filtered_frames_info.iterrows():
        
        x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
        #extracted_text_box = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
        
        print("{:<5} {:<10} {:<30} {:<20} {:<20} {:<7} {:<7} {:<7} {:<7}".format(row_frame['seq'], row_frame['model'], row_frame['father'], row_frame['label'], row_frame['reference'], row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1'] ))
        
        # Divida o texto por nova linha e mantenha apenas a última parte (assume que o valor está sempre no final)
        #value = extracted_text_box.split('\n')[-1]
        # Remova qualquer espaço em branco à esquerda ou à direita
        #value = value.strip()
        
        for index_field, row_field in filtered_sframe_fields_info.iterrows():
            
            print("{:<5} {:<10} {:<30} {:<20} {:<20}".format(row_field['seq'], row_field['model'], row_field['father'], row_field['label'], row_field['reference']))


### data_dados_complementares, data_outras_informacoes, data_observacao

In [26]:
# data_prestador
data_prestador = {}
father_value = "2_frame_cnpj_prestador"
section = "2. PRESTADOR DE SERVIÇO"


extract_fields_frame(modelo, father_value, section)

2. PRESTADOR DE SERVIÇO
14    mage_1     2_section_prestador_servico    2_frame_cnpj_prestador nan                  0.0     380.0   550.0   555.0  
15    mage_1     2_frame_cnpj_prestador         cpf_cnpj_sem_mascara CPF/CNPJ:           
16    mage_1     2_frame_cnpj_prestador         cpf_cnpj_com_mascara CPF/CNPJ:           
17    mage_1     2_frame_cnpj_prestador         telefone             Telefone:           


In [27]:
# data_CNAE
data_CNAE = {}
father_value = "4_frame_cnae_itens_servico"
section = "6. CNAE e Item da Lista de Serviços"


extract_fields_frame(modelo, father_value, section)

6. CNAE e Item da Lista de Serviços
44    mage_1     4_section_servicos_totais      4_frame_cnae_itens_servico nan                  0.0     1844.0  2067.0  1943.0 
45    mage_1     4_frame_cnae_itens_servico     cnae                 CNAE                
46    mage_1     4_frame_cnae_itens_servico     item_lista_servicos  Itens da Lista de Serviços


In [28]:
# data_dados_complementares
data_dados_complementares = {}
father_value = "5_frame_dados_complementares"
section = "8. DADOS COMPLEMENTARES"

extract_fields_frame(modelo, father_value, section)

8. DADOS COMPLEMENTARES
65    mage_1     6_section_inf_complementares_criticas 5_frame_dados_complementares nan                  148.0   2273.0  1925.0  2377.0 
66    mage_1     5_frame_dados_complementares   dados_complementares DADOS COMPLEMENTARES


In [29]:
# data_observacao
data_observacao = {}
father_value = "5_frame_observacao"
section = "10. OBSERVACOES"

extract_fields_frame(modelo, father_value, section)

10. OBSERVACOES
74    mage_1     6_section_inf_complementares_criticas 5_frame_observacao   nan                  148.0   2521.0  1922.0  2676.0 
75    mage_1     5_frame_observacao             observação           OBSERVAÇÃO:         


### Informacoes dos DFs

In [ ]:
# Informaçao sobre os DF sendo utilizados
sections_info
frames_info
sframe_fields_info
field_boxes_info

frames_info.head()

sframe_fields_info.head()

field_boxes_info.head()

field_boxes_info.info

## 5. Operaçoes com arquivos e diretorios & Logging

<mark> Config </mark>
- root_pdf_path  (local inicial dos PDFs)

- image_resized_path (local da imagem que foi convetida e resized)

- nf_processada_path (local onde ficam os PDFs processados)


<mark> Criados no processo </mark>

- image_2work (arquivo de imagem resized criado)

- name_image_2work (path e nome da imagem resized criada)

- document_path (path e nome do PDF utilizado para tratamento)

In [ ]:
# mover PDF para diretorio de processados
source_path = document_path
destination_path = os.path.join(f'{nf_processada_path}/{str(doc2convert)}')
shutil.move(source_path, destination_path)

In [ ]:
# ***DELETAR**** o arquivo de imagem gerado
image_path_to_delete = name_image_2work
os.remove(image_path_to_delete)

In [ ]:
# voltar (mover) PDF para diretorio para ser processado
source_path = destination_path

destination_path = document_path

shutil.move(source_path, destination_path)

In [ ]:
# Processo para iterar estrutura de arquivos, diretorios
for roots, directories, documents in os.walk(image_resized_path):
    print(f'\n {documents} {name_image_2work}\n\n')
    #if documents == :
        

In [ ]:
# 7. Buscar o primeiro documento do diretorio EXTERNO para processar
source_directory = root_external_pdf_path
destination_directory = root_pdf_path

# Verifique se o diretório de destino está vazio
if os.listdir(destination_directory):
    raise Exception("O diretório de destino não está vazio!")

for roots, directories, documents in os.walk(source_directory):
    # Filtre os documentos para incluir apenas aqueles com extensão .pdf
    pdf_files = [doc for doc in documents if doc.lower().endswith('.pdf')]
    
    if pdf_files: # Verifique se há algum arquivo PDF no diretório
        first_pdf_file = pdf_files[0] # Obtenha o primeiro arquivo PDF
        source_path = os.path.join(roots, first_pdf_file) # Construa o caminho completo para o primeiro arquivo PDF
        print(f"Found the first PDF file: {source_path}")
        
        # Mova o arquivo para o diretório de destino
        shutil.move(source_path, destination_directory)
        
        break # Saia do loop após encontrar o primeiro arquivo PDF

### Logging

LOG - Single File vs. Separate Files

1. Single File vs. Separate Files
Single File: Suitable if the volume of logs is manageable and you want a consolidated view of all activities.
Separate Files: Useful if you want to organize logs by day or by a specific module or process. This can help in troubleshooting specific issues and managing large volumes of logs.
2. Concise vs. Detailed Logs
You can create different log levels to handle concise and detailed logging:

Concise Log: Use the INFO level to log only essential information, such as process start and end times, success messages, etc.
Detailed Log: Use the DEBUG level to log detailed information, like variable values, intermediate steps, etc.
Example with Python's logging Module
Here's an example of how you might set up logging to handle both separate files and different log levels:

In [ ]:
import logging

# Configure logging
log_file_path = 'novo_modelo/execucao_pipeline_extracao_pdf.log'
logging.basicConfig(
    filename=log_file_path,
    filemode='a', # Append mode
    level=logging.INFO, # Log level (e.g., INFO, WARNING, ERROR)
    format='%(asctime)s - %(levelname)s - %(message)s', # Log message format
    datefmt='%Y-%m-%d %H:%M:%S' # Date format
)

In [ ]:
logging.info("Pipeline iniciado.")

In [ ]:
# Example usage of the logger
logging.info("Pipeline started.")
logging.info("Processing file: %s", first_pdf_file_path)
logging.warning("File has more than one page.")
logging.error("An error occurred while processing file: %s", first_pdf_file_path)

By changing the log level, you can control the verbosity of the logs. If you want separate log files for each day, the above code will create a new file every day based on the date.

Remember, logging strategy should align with your application's needs, including debugging, auditing, and compliance requirements. Make sure to review and test your logging setup to ensure it meets your needs.

In [ ]:
import logging
import datetime

# Get the current date for filename
current_date = datetime.datetime.now().strftime("%Y-%m-%d")
log_filename = f"log_{current_date}.txt"

# Set up logging
logging.basicConfig(
    filename=log_filename,
    level=logging.DEBUG,  # Log everything (change to INFO for concise logs)
    format="%(asctime)s - %(levelname)s - %(message)s",
)

# Example of concise log (INFO level)
logging.info("Processing started.")

# Example of detailed log (DEBUG level)
logging.debug("Variable x value is 10.")

# ... rest of your code ...

# Handle exceptions with logging
try:
    # Your code that might raise an exception
    pass
except Exception as e:
    logging.error(f"An error occurred: {e}")


## 6. Teste e codigos para analisar

### Tesseract

<mark> Testar esta configuraçao de Tesseract </mark>

In [ ]:
def detect(cropped_frame, is_number = False):
    if (is_number):
        text = pytesseract.image_to_string(cropped_frame,
                                           config ='-c tessedit_char_whitelist=0123456789 --psm 10 --oem 2')
    else:
        text = pytesseract.image_to_string(cropped_frame, config='--psm 10')        
        
    return text


### Draw Boxes with matplotlib

Using the Matplotlib library to draw boxes on an image is entirely feasible and provides more control over the appearance and customization of the drawings. Below, I'll provide an example of how you can draw boxes on an image using Matplotlib.

First, make sure you have the necessary libraries installed:

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def draw_boxes(image, boxes_info, color='r'):
    # Create a figure and axis
    fig, ax = plt.subplots(1)
    
    # Display the image
    ax.imshow(image)

    # Draw the boxes
    for index, row in boxes_info.iterrows():
        x0, y0, x1, y1 = row['x0'], row['y0'], row['x1'], row['y1']
        # Create a Rectangle patch
        rect = patches.Rectangle((x0, y0), x1 - x0, y1 - y0, linewidth=1, edgecolor=color, facecolor='none')
        # Add the patch to the Axes
        ax.add_patch(rect)

    plt.show()


You can call this function with your image object and DataFrame containing the coordinates of the boxes to be drawn:

In [ ]:
draw_boxes(image, boundaries_info, color='r')
draw_boxes(image, sections_info, color='g')


You can customize the appearance of the boxes by changing the parameters of the patches.Rectangle function, such as line width, edge color, and face color.

Using Matplotlib in this way allows you to have more control over the appearance of the plot, and you can easily combine it with other Matplotlib features to create complex visualizations.


### Tecnicas de manitpulacao de imagens

In [ ]:
from io import StringIO

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

def convert_pdf_to_string(file_path):

	output_string = StringIO()
	with open(file_path, 'rb') as in_file:
	    parser = PDFParser(in_file)
	    doc = PDFDocument(parser)
	    rsrcmgr = PDFResourceManager()
	    device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
	    interpreter = PDFPageInterpreter(rsrcmgr, device)
	    for page in PDFPage.create_pages(doc):
	        interpreter.process_page(page)

	return(output_string.getvalue())


2407-106 TERE SÃO PEDRO.pdf
                
def convert_title_to_filename(title):
    filename = title.lower()
    filename = filename.replace(' ', '_')
    return filename


def split_to_title_and_pagenum(table_of_contents_entry):
    title_and_pagenum = table_of_contents_entry.strip()
    
    title = None
    pagenum = None
    
    if len(title_and_pagenum) > 0:
        if title_and_pagenum[-1].isdigit():
            i = -2
            while title_and_pagenum[i].isdigit():
                i -= 1

            title = title_and_pagenum[:i].strip()
            pagenum = int(title_and_pagenum[i:].strip())
        
    return title, pagenum

In [ ]:
import io
import os
import sys
import time

import fitz
import PySimpleGUI as sg

print(fitz.__doc__)

if not tuple(map(int, fitz.version[0].split("."))) >= (1, 18, 18):
    raise SystemExit("require PyMuPDF v1.18.18+")

dimlimit = 0  # 100  # each image side must be greater than this
relsize = 0  # 0.05  # image : image size ratio must be larger than this (5%)
abssize = 0  # 2048  # absolute image size limit 2 KB: ignore if smaller
imgdir = "output"  # found images are stored in this subfolder

if not os.path.exists(imgdir):  # make subfolder if necessary
    os.mkdir(imgdir)


def recoverpix(doc, item):
    xref = item[0]  # xref of PDF image
    smask = item[1]  # xref of its /SMask

    # special case: /SMask or /Mask exists
    if smask > 0:
        pix0 = fitz.Pixmap(doc.extract_image(xref)["image"])
        if pix0.alpha:  # catch irregular situation
            pix0 = fitz.Pixmap(pix0, 0)  # remove alpha channel
        mask = fitz.Pixmap(doc.extract_image(smask)["image"])

        try:
            pix = fitz.Pixmap(pix0, mask)
        except:  # fallback to original base image in case of problems
            pix = fitz.Pixmap(doc.extract_image(xref)["image"])

        if pix0.n > 3:
            ext = "pam"
        else:
            ext = "png"

        return {  # create dictionary expected by caller
            "ext": ext,
            "colorspace": pix.colorspace.n,
            "image": pix.tobytes(ext),
        }

    # special case: /ColorSpace definition exists
    # to be sure, we convert these cases to RGB PNG images
    if "/ColorSpace" in doc.xref_object(xref, compressed=True):
        pix = fitz.Pixmap(doc, xref)
        pix = fitz.Pixmap(fitz.csRGB, pix)
        return {  # create dictionary expected by caller
            "ext": "png",
            "colorspace": 3,
            "image": pix.tobytes("png"),
        }
    return doc.extract_image(xref)


fname = sys.argv[1] if len(sys.argv) == 2 else None
if not fname:
    fname = sg.PopupGetFile("Select file:", title="PyMuPDF PDF Image Extraction")
if not fname:
    raise SystemExit()

t0 = time.time()
doc = fitz.open(fname)

page_count = doc.page_count  # number of pages

xreflist = []
imglist = []
for pno in range(page_count):
    sg.QuickMeter(
        "Extract Images",  # show our progress
        pno + 1,
        page_count,
        "*** Scanning Pages ***",
    )

    il = doc.get_page_images(pno)
    imglist.extend([x[0] for x in il])
    for img in il:
        xref = img[0]
        if xref in xreflist:
            continue
        width = img[2]
        height = img[3]
        if min(width, height) <= dimlimit:
            continue
        image = recoverpix(doc, img)
        n = image["colorspace"]
        imgdata = image["image"]

        if len(imgdata) <= abssize:
            continue
        if len(imgdata) / (width * height * n) <= relsize:
            continue

        imgfile = os.path.join(imgdir, "img%05i.%s" % (xref, image["ext"]))
        fout = open(imgfile, "wb")
        fout.write(imgdata)
        fout.close()
        xreflist.append(xref)

t1 = time.time()
imglist = list(set(imglist))
print(len(set(imglist)), "images in total")
print(len(xreflist), "images extracted")
print("total time %g sec" % (t1 - t0))

In [ ]:
"""
A basic text-to-PDF converter
--------------------------------------------------------------------------------
License: GNU GPL V3
(c) 2018 Jorj X. McKie

Usage
-----
python convert.py input.txt
"""

import sys
import fitz

assert len(sys.argv) == 2, "usage: python %s text.file" % (sys.argv[0],)
ifn = sys.argv[1]
ofn = "output1.pdf"

width, height = fitz.paper_size("a4")
fontsz = 10
lineheight = fontsz * 1.2

nlines = int((height - 108.0) / lineheight)

sourcefile = open(ifn)
line_ctr = 0  # page line counter
total_ctr = 0  # total line counter
out_ctr = 0  # count output lines
out_buf = ""  # text of one page

doc = fitz.open()


def page_out(b):
    page = doc.new_page(width=width, height=height)
    return page.insert_text(
        (50, 72),
        text=b,
        fontsize=fontsz,
    )


while True:
    line = sourcefile.readline()
    if line == "":
        break
    out_buf += line
    line_ctr += 1
    total_ctr += 1
    if line_ctr == nlines:
        out_ctr += page_out(out_buf)
        out_buf = ""
        line_ctr = 0

if len(out_buf) > 0:
    out_ctr += page_out(out_buf)

print("PDF conversion results for file '%s':" % (ifn,))
print(out_ctr, "lines read,", total_ctr, "lines written,", nlines, "lines per page.")
print(ofn, "contains", len(doc), "pages.")

# Now add a header and footer to each page
hdr_fontsz = 16
ftr_fontsz = 8
blue = fitz.pdfcolor["blue"]
pspace = 500

for page in doc:
    footer = "%i (%i)" % (page.number + 1, len(doc))  # footer text
    plen_ftr = fitz.get_text_length(footer, fontname="Helvetica", fontsize=ftr_fontsz)
    page.insert_text(
        (50, 50), ifn, color=blue, fontsize=hdr_fontsz  # header = input filename
    )
    page.draw_line(
        fitz.Point(50, 60),
        fitz.Point(50 + pspace, 60),  # line below hdr
        color=blue,
        width=0.5,
    )
    page.draw_line(
        fitz.Point(50, height - 33),  # line above footer
        fitz.Point(50 + pspace, height - 33),
        color=blue,
        width=0.5,
    )
    page.insert_text(
        (50 + pspace - plen_ftr, height - 33 + ftr_fontsz * 1.2),  # insert footer
        footer,
        fontsize=ftr_fontsz,
        color=blue,
    )
    page.clean_contents()

doc.set_metadata(
    {
        "creationDate": fitz.get_pdf_now(),
        "modDate": fitz.get_pdf_now(),
        "creator": "convert.py",
        "producer": "PyMuPDF %s" % fitz.VersionBind,
        "title": "Content of file " + ifn,
        "subject": "Demonstrate methods new_page, insert_text and draw_line",
        "author": "Jorj McKie",
    }
)
doc.subset_fonts()
doc.ez_save(ofn, garbage=4, pretty=True)
doc.close()
